# 模板模式

编写优秀代码的一个要素是避免冗余。在面向对象编程中，方法和函数是我们用来避免编写 冗余代码的重要工具。回想第15章中的sorted()例子。sorted()函数非常通用，可使用任意键 来对多种数据结构（列表、元组和命名元组等）进行排序。这是一个良好函数的定义。

sorted()这样的函数属于理想的案例。现实中，我们没法始终写出100%通用的代码。许多 算法都有一些（但并非全部）通用步骤。广度优先搜索（Breadth-First Search，BFS）和深度优先 搜索（Depth-First Search，DFS）是其中不错的例子，这两个流行的算法应用于图搜索问题。起 初，我们提出独立实现两个算法（文件graph.py）。函数bfs()和dfs()在start和end之间存在 一条路径时返回一个元组(True, path)；如果路径不存在，则返回(False, path)（此时， path为空）。

In [ ]:
def bfs(graph, start, end):
    path = []
    visited = [start]
    while visited:
        current = visited.pop(0)
        if current not in path:
            path.append(current)
            if current == end:
                print(path)
                return (True, path)
            # 两个顶点不相连，则跳过
            if current not in graph:
                continue
        visited = visited + graph[current]
    return (False, path)

In [ ]:
def dfs(graph, start, end):
        path = []
        visited = [start]
        while visited:
                current = visited.pop(0)
                if current not in path:
                        path.append(current)
                        if current == end:
                                return (True, path)
                        # 两个顶点不相连，则跳过
                        if current not in graph:
                                continue
                visited = graph[current] + visited
        return (False, path)


注意两个算法之间的相似点。仅有一处不同（已加粗），其余部分完全相同。稍后我们再回 来讨论这个问题。

先使用Wikipedia提供的图（请参考网页[t.cn/RqrBp3p]） 来测试算法。为了简化，假设该图 是有向的。这意味着只能朝一个方向移动，我们可以检测如何从Frankfurt到Mannheim，而不是另 一个方向。

可以使用列表的字典结构来表示这个有向图。每个城市是字典中的一个键，列表的内容是从 该城市始发的所有可能H的地。叶子顶点的城市（例如，Erfurt）使用一个空列表即可（无目的地）。

In [ ]:
def main():
    graph = {
        'Frankfurt':  ['Mannheim', 'Wurzburg', 'Kassel'],
        'Mannheim':   ['Karlsruhe'],
        'Karlsruhe':  ['Augsburg'],
        'Augsburg':   ['Munchen'],
        'Wurzburg':   ['Erfurt', 'Nurnberg'],
        'Nurnberg':   ['Stuttgart', 'Munchen'],
        'Kassel':     ['Munchen'],
        'Erfurt':     [],
        'Stuttgart':  [],
        'Munchen':    []
        }

    bfs_path = bfs(graph, 'Frankfurt', 'Nurnberg')
    dfs_path = dfs(graph, 'Frankfurt', 'Nurnberg')
    print('bfs Frankfurt-Nurnberg: {}'.format(bfs_path[1] if bfs_path[0] else 'Not found'))
    print('dfs Frankfurt-Nurnberg: {}'.format(dfs_path[1] if dfs_path[0] else 'Not found'))

    bfs_nopath = bfs(graph, 'Wurzburg', 'Kassel')
    print('bfs Wurzburg-Kassel: {}'.format(bfs_nopath[1] if bfs_nopath[0] else 'Not found'))
    dfs_nopath = dfs(graph, 'Wurzburg', 'Kassel')
    print('dfs Wurzburg-Kassel: {}'.format(dfs_nopath[1] if dfs_nopath[0] else 'Not found'))


从性质来看，结果并不能表明什么，因为DFS和BFS不能很好地处理加权图（权重完全被忽 略了）。处理加权图更好的算法是（Dijkstra的）最短路径优先算法、Bellman-Ford算法和A*算法 等。然而，我们仍然希望按打算的那样遍历图。我们期望的算法输出是一个城市列表，这些城市 是在搜索从Frankfurt到Nurnberg的路径时访问过的。

>> python3 graph.py
bfs Frankfurt-Nurnberg: ['Frankfurt', 'Mannheim', 'Wurzburg', 'Kassel', 'Karlsruhe',
'Erfurt', 'Nurnberg']
dfs Frankfurt-Nurnberg: ['Frankfurt', 'Mannheim', 'Karlsruhe', 'Augsburg', 'Munchen', 'Wurzburg', 'Erfurt', 'Nurnberg']
bfs Wurzburg-Kassel: Not found dfs Wurzburg-Kassel: Not found

结果看起来没问题。BFS按广度进行遍历，DFS则按深度进行遍历，两个算法都没返回任何 非期望的结果。这样不错，但我们的代码仍然有一个问题，那就是冗余。两个算法之间仅有一处 不同，但其余代码都写了两遍。对于这个问题我们能做点什么吗？

是的！这个问题可以通过模板设计模式（Template design pattern）来解决。这个模式关注的 是消除代码冗余，其思想是我们应该尤需改变算法结构就能重新定义一个算法的某些部分。为了 避免重复而进行必要的重构之后，我们来看看代码会变成什么样子（文件graph_template.py）。

In [ ]:
def traverse(graph, start, end, action):
    path = []
    visited = [start]
    while visited:
        current = visited.pop(0)
        if current not in path:
            path.append(current)
            if current == end:
                return (True, path)
            # skip vertices with no connections
            if current not in graph:
                continue
            visited = action(visited, graph[current])
        return (False, path)

def extend_bfs_path(visited, current):
    return visited + current

def extend_dfs_path(visited, current):
    return current + visited


不再有bfs()和dfs()两个函数，我们将代码重构为使用单个traverse()函数。traverse() 函数实际上是一个模板函数。它接受一个参数action，该参数是一个“知道”如何延伸路径的 函数。根据要使用的算法，我们可以传递extend_bfs_path()或extend_dfs_path()作为目标动作。

你也许会争论说，通过在traverse()内部添加一个条件来检测应该使用哪个遍历算法，也能达到相同的结果。下面的代码展示了这个思路（文件graph_template_slower.py）。